In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# 先定义一些工具：Tools

- 可以是一个函数或三方 API
- 也可以把一个 Chain 或者 Agent 的 run()作为一个 Tool


## 搜索工具

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import Tool, tool

search = TavilySearchResults()

tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
    ),
]

## 日期转换工具

In [7]:
import calendar
import dateutil.parser as parser
from datetime import date

# 自定义工具
@tool("weekday")
def weekday(date_str: str) -> str:
    """Convert date to weekday name"""
    d = parser.parse(date_str)
    return calendar.day_name[d.weekday()]

tools += [weekday]

In [8]:
weekday("8/30/1979")

'Thursday'

# create_react_agent

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

In [21]:
tools

[Tool(name='Search', description='useful for when you need to answer questions about current events', func=<bound method BaseTool.run of TavilySearchResults()>),
 StructuredTool(name='weekday', description='weekday(date_str: str) -> str - Convert date to weekday name', args_schema=<class 'pydantic.main.weekdaySchemaSchema'>, func=<function weekday at 0x10793f760>),
 StructuredTool(name='weekday', description='weekday(date_str: str) -> str - Convert date to weekday name', args_schema=<class 'pydantic.main.weekdaySchemaSchema'>, func=<function weekday at 0x11840b490>)]

In [22]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [23]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [24]:
agent_executor.invoke({"input": "周杰伦的生日是星期几？"})



> Entering new AgentExecutor chain...
I need to find out the date of Jay Chou's (周杰伦) birthday and then use the weekday function to determine the day of the week.
Action: Search
Action Input: 周杰伦 生日
[{'url': 'https://baike.baidu.com/item/周杰伦/129156', 'content': '周杰伦\n0\n目录\n周杰伦早年经历\n周杰伦演艺经历\n周杰伦个人生活\n周杰伦主要作品\n周杰伦音乐专辑\n周杰伦音乐单曲\n周杰伦为他人创作\n周杰伦演唱会记录\n周杰伦参演电影\n叱咤风云2021-1-15\n练爱ING2020\n极限特工42019\n惊天魔盗团22016-06-10\n天台爱情2013-07-11\n逆战2012-01-19\n青蜂侠2011-02-08\n苏乞儿2010-02-09\n刺陵2009-12-09\n大灌篮2008-02-07\n不能说的秘密2007-07-31\n满城尽带黄金甲2006-12-14\n头文字D2005-06-24\n自行我路2005-03-22\n寻找周杰伦2003-11-13\n周杰伦参演电视剧\n熊猫人2010-01-17\n蓝星2003\n星情花园2002\n百里香煎鱼1998\n周杰伦综艺节目\n发起人[89]\n担任导师[74]\n担任嘉宾\n担任导师\n担任导师（与庾澄庆、那英、汪峰）\n观魔评审\n科学助教\n担任主讲嘉宾\n主持人（与杜国璋、小麦、雪糕、乙黑绘理）\n周杰伦导演作品\n周杰伦编剧作品\n周杰伦出版作品\n周杰伦监制作品\n周杰伦配音作品\n周杰伦社会活动\n周杰伦公益活动\n周杰伦担任大使\n周杰伦其它领域\n周杰伦获奖记录\n周杰伦人物评价\n周杰伦《最伟大作品》MV中[【五大名贵钢琴】你知道吗？\n2022年周董的专辑《最伟大的作品》，又一个华语歌曲的天花板。其中《最伟大的作品》、《还在流浪》、《错过的烟火》三首歌曲MV中使用了5台名贵钢琴，总价值超过千万，这其中包括施坦威、Baldwin鲍德温、老式里特米勒，几台钢琴巧妙的放在了不同的场景下，表现了

{'input': '周杰伦的生日是星期几？', 'output': '周杰伦的生日是星期四。'}

# create_structured_chat_agent

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system = '''
Respond to the human as helpfully and accurately as possible. 
You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! 
ALWAYS respond with a valid json blob of a single action;
Always think and respond with Chinese.

Use tools if necessary. 
Respond directly if appropriate. 
Format is Action:```$JSON_BLOB```then Observation.

'''

human = '''{input}

{agent_scratchpad}

(reminder to respond in a JSON blob no matter what)
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human),
    ]
)

In [35]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_openai import OpenAI

# from langchain import hub
# prompt = hub.pull("hwchase17/structured-chat-agent")

llm = OpenAI()
agent = create_structured_chat_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)